# Load and test trained model

## Load libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from enduro_lstm import *

In [2]:
device = conf_cuda(False)

Selected CPU


In [3]:
device

device(type='cpu')

In [4]:
device.type

'cpu'

## Set configurations

In [108]:
data_path = r"../1-generate/data/"
match = 50
use_cuda = False
zigzag = False

## Load trained model

In [85]:
newpath = 'models/report/play_m45to49_f1to1000_epoch10000_H200/' + 'play_m45to49_f1to1000_epoch10000_H200'

In [86]:
hidden_neurons = 200
load_checkpoint = True

In [87]:
if zigzag:
    output_size = 2
else:
    output_size = 9

In [88]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model

In [89]:
if load_checkpoint:
    model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
    model = load_checkpoint(model, newpath)
else:
    model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
    model.load_state_dict(torch.load(newpath, map_location=device))
model.eval()

=> loading checkpoint 'models/report/play_m45to49_f1to1000_epoch10000_H200/play_m45to49_f1to1000_epoch10000_H200'


Model(
  (lstm): LSTM(20400, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=9, bias=True)
  (sigmoid): Sigmoid()
)

In [59]:
model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
torch.load(newpath, map_location=device)
model.eval()

Model(
  (lstm): LSTM(20400, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=9, bias=True)
  (sigmoid): Sigmoid()
)

## Prepare cell with trained model 

In [90]:
lstmcell = nn.LSTMCell(20400, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
sigmoid = nn.Sigmoid()

In [91]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [92]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [93]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([800, 20400])
torch.Size([800, 200])
torch.Size([800])
torch.Size([800])
torch.Size([9, 200])
torch.Size([9])


## Testing outputs of model

In [94]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [109]:
num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, match)

Successfully loaded NPZ.


In [110]:
X_train = frames.reshape(1, len(frames) ,-1)/255

In [111]:
ACTIONS_LIST

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [112]:
actions

array([0, 0, 0, ..., 1, 1, 1])

In [113]:
action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
Y_train = np.array(action_one_hot)
Y_train = Y_train.reshape(1, len(Y_train), -1)

In [114]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [115]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)
output = []
for i in range(1000):
    step_input = X_train[0][i]
    step_input = step_input.reshape(1, -1)
    hx, cx = lstmcell(step_input, (hx, cx))
    out = linear(hx)
    out = sigmoid(out)
    output.append(out)

In [116]:
Y_train = Y_train.reshape(-1, len(ACTIONS_LIST))

In [117]:
acertou = 0
errou = 0
for i in range(1000):
    if torch.argmax(Y_train[i]) == torch.argmax(output[i]):
        acertou += 1
    else:
        errou += 1 

In [118]:
Y_train

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])

In [119]:
acertou

486

In [120]:
errou

514

In [121]:
acertou/(acertou + errou)

0.486

## Play Gym Enduro

In [50]:
import gym
import time
from PIL import Image

In [51]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [52]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [53]:
sleep_time = 0.05

In [53]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

while(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx, cx = lstmcell(frame, (hx, cx))
    out = linear(hx)
    action = sigmoid(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-54-1a82cc127eec>", line 10, in <module>
    time.sleep(sleep_time)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/genericpath.py", line 19, in exists
    os.stat(path)
FileNotFoundError: [Errno 2] No such file or directory: '<ipython-input-54-1a82cc127eec>'

During handling of the above exception, another exception occurred:

Traceba

TypeError: object of type 'NoneType' has no len()